In [3]:
#---------------------------------------------------------------------------------------------------------------
#  描写用の関数
#---------------------------------------------------------------------------------------------------------------
import random
import cartopy.crs as ccrs
import cartopy.feature as cfea
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.gridspec as gridspec
import matplotlib.colors as mcolors
from matplotlib.colors import ListedColormap
from matplotlib.colors import CSS4_COLORS
from matplotlib.patches import Rectangle
import cartopy.io.shapereader as shapereader

#---------------------------------------------------------------------------------------------------------------
#  流域4色塗分け問題
#---------------------------------------------------------------------------------------------------------------
import pulp
import pandas as pd
from ortoolpy import model_min, addbinvars, addvals
from pulp import lpSum

def get_unique_non_masked_values(values):
    result = []
    for value in np.unique(values):
        if value is not np.ma.masked and not np.isnan(value):
            result.append(value)
    result = [x for x in result if x != -1.0]
    return result

def get_adjacent(arr, basin_num):
    neighbors = []
    rows, cols = arr.shape
    directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
    for row in range(rows):
        for col in range(cols):
            if int(arr[row, col]) == int(basin_num):
                for dr, dc in directions:
                    r, c = row + dr, col + dc
                    if 0 <= r < rows and 0 <= c < cols:
                        if int(arr[r,c]) != int(basin_num):
                            neighbors.append(int(arr[r, c]))
                        
    unique_neighbors = get_unique_non_masked_values(neighbors)
    return unique_neighbors

def four_color_problem(Rivnum_masked_array, color_code):
    basin_lst = get_unique_non_masked_values(Rivnum_masked_array)
    
    # make data
    df = pd.DataFrame([
    (i, int(basin_lst[i]), j) 
    for i in range(len(basin_lst)) 
    for j in range(len(color_code))
    ], columns=['code', 'basin_num', 'color'])
    
    # optimization
    m = model_min()  # 数理モデル(1)
    addbinvars(df)  # 変数表(2)
    for i in range(len(basin_lst)):
        m += lpSum(df[df.code == i].Var) == 1  # 1県1色(3)
        for j in get_adjacent(Rivnum_masked_array, int(basin_lst[i])):
            for k in range(len(color_code)):  # 隣接県を違う色に(4)
                m += lpSum(df[((df.code == i)  | (df.basin_num == int(j))) & (df.color == k)].Var) <= 1
    m.solve(pulp.PULP_CBC_CMD(msg=0))  # 求解(5)
    addvals(df)  # 結果設定(6)
    cols = df[df.Val > 0].color.apply(color_code.__getitem__).reset_index(drop=True)
    return df
    
def pulp_cmap(Rivnum_masked_array, color_code):
    # ユニークな流域の数をカウント
    unique_values = get_unique_non_masked_values(Rivnum_masked_array)
    num_colors = len(unique_values)
    
    df = four_color_problem(Rivnum_masked_array, color_code)
    opt_df = df[df.Val>0]
    color_list = opt_df['color'].tolist()
    colors_list = [color_code[i] for i in color_list]

    # HTMLカラーコードに変化してlisted colormap を作成
    cmap = ListedColormap(colors_list)

    # [辞書作る] key=流域ID, value=0から数えたindex　
    index_dict = {}
    for i, value in enumerate(unique_values):
        index_dict[value] = i

    return num_colors, index_dict, cmap

def get_index(val, index_dict):
    return index_dict.get(val, -1)  # nan or unexpected values are mapped to -1

In [7]:
import numpy as np
path = '/mnt/c/Users/tsimk/Downloads/research/H08/additional_data/matsumura/takeover/subwatersheds_150000_vanilla.bin'
data = np.fromfile(path, dtype='float32').reshape(2160, 4320)

nan
nan


In [ ]:
import random
def plot_basin(rivnum):
    # highlight basin
    unique_rivnum = np.unique(rivnum)
    filtered = [i for i in unique_rivnum if i != 0]
    
    # cmap
    color_code = ['#4169e1', '#fffacd', '#c71585', '#00fa9a', '#ba55d3', '#48d1cc', '#ffc0cb', '#ffa07a']
    random.seed(7)
    random.shuffle(color_code)
    
    color_masked = np.ma.masked_where(~np.isin(rivnum, filtered), rivnum)
    color_filled = color_masked.filled(-1)
    num_colors, index_dict, cmap = pulp_cmap(color_filled, color_code)
    
    # data
    data_masked = np.ma.masked_where(rivnum >= 1e20, rivnum)
    data_filled = data_masked.filled(-1)
    indexed_data = np.vectorize(get_index)(data_filled, index_dict)
    indexed_data_masked = np.ma.masked_where(indexed_data<0, indexed_data)
    
    # extent
    projection = ccrs.PlateCarree()
    fig = plt.figure(figsize=(50,50))
    ax = plt.subplot(projection=projection)
    ax.set_extent(extent, projection)
    #ax.add_feature(cfea.OCEAN, color='#BBBBBB')
    #ax.add_feature(cfea.LAKES, color='#BBBBBB')
    ax.coastlines(zorder=0)
    img = ax.imshow(indexed_data_masked,
                    origin='upper',
                    extent=extent,
                    transform=projection,
                    cmap=cmap,
                    vmin=-0.5, vmax=num_colors-0.5,
                    alpha=1.0)
    
plot_basin(data)

In [ ]:
output_file = "./colormap.txt"
with open(output_file, "w") as f:
    for i, color in enumerate(colors):
        r, g, b, a = (color[:3] * 255).astype(int)  # RGB値を0-255スケールに変換
        f.write(f"{i} {r} {g} {b} {a}\n")  # フォーマット: "値 R G B A"
print(f"Colormap saved to {output_file}")

In [ ]:
import numpy as np

# QGIS用カラーテーブルを作成する関数
def save_qgis_colormap(custom_cmap, output_file, num_colors=256):
    """
    custom_cmapからQGIS用のカラーテーブルを作成し、テキストファイルに保存する。
    
    Args:
        custom_cmap: matplotlib.colors.ListedColormap のインスタンス
        output_file: 保存先ファイル名（例: "colormap.txt"）
        num_colors: カラーマップの分割数（デフォルト: 256）
    """
    # カラーマップを num_colors に分割して取得
    colors = custom_cmap(np.linspace(0, 1, num_colors))  # RGBA (0-1スケール)

    # ファイルに保存
    with open(output_file, "w") as f:
        for i, color in enumerate(colors):
            r, g, b, a = (color[:3] * 255).astype(int)  # RGBを0-255スケールに変換
            f.write(f"{i} {r} {g} {b} {a}\n")  # フォーマット: "値 R G B A"
    print(f"Colormap saved to {output_file}")

# 使用例
# custom_cmap を既に作成済みの場合
output_file = "colormap.txt"
save_qgis_colormap(custom_cmap, output_file)
